In [1]:
import pandas as pd
import numpy as np

### save 10% for heldout test, use 90% for cv

In [32]:
df = pd.read_csv("./data/hw1_train.csv", index_col="ID")
split_size = int(df.shape[0]/10)

In [41]:
np.random.seed(0)
test_idx = np.random.choice(df.index, size=split_size)
rest = np.array([i for i in df.index if i not in test_idx])
val_idx = np.random.choice(rest, size=split_size)
train_idx = np.array([i for i in rest if i not in val_idx])

In [43]:
df.loc[train_idx].to_csv("./data/hw1_train_real.csv")